<a href="https://colab.research.google.com/github/AlastairBss/Minecraft-Servers/blob/main/Copy_of_Untitled14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **k-Nearest Neighbour (KNN)**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X = iris.data
y = iris.target
names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

predictions = knn.predict(X_test)

print("=== Prediction Results ===")
for actual, pred in zip(y_test, predictions):
    if actual == pred:
        print(f"✅ Correct: {names[pred]}")
    else:
        print(f"❌ Wrong: Predicted {names[pred]}, Actual {names[actual]}")

accuracy = knn.score(X_test, y_test)
print(f"\nAccuracy: {accuracy * 100:.2f}%")


=== Prediction Results ===
✅ Correct: versicolor
✅ Correct: setosa
✅ Correct: virginica
✅ Correct: versicolor
✅ Correct: versicolor
✅ Correct: setosa
✅ Correct: versicolor
✅ Correct: virginica
✅ Correct: versicolor
✅ Correct: versicolor
✅ Correct: virginica
✅ Correct: setosa
✅ Correct: setosa
✅ Correct: setosa
✅ Correct: setosa
✅ Correct: versicolor
✅ Correct: virginica
✅ Correct: versicolor
✅ Correct: versicolor
✅ Correct: virginica
✅ Correct: setosa
✅ Correct: virginica
✅ Correct: setosa
✅ Correct: virginica
✅ Correct: virginica
✅ Correct: virginica
✅ Correct: virginica
✅ Correct: virginica
✅ Correct: setosa
✅ Correct: setosa

Accuracy: 100.00%


### **ARTIFICIAL NEURAL NETWORK (ANN)**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = MLPClassifier(hidden_layer_sizes=(5, 3), max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", round(accuracy_score(y_test, y_pred) * 100, 2), "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 91.11 %

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.69      0.82        13
           2       0.76      1.00      0.87        13

    accuracy                           0.91        45
   macro avg       0.92      0.90      0.89        45
weighted avg       0.93      0.91      0.91        45



/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


### **CANDIDATE**

In [ ]:
import pandas as pd

def load_data(file_path):
    return pd.read_csv(file_path)

def more_general(h1, h2):
    return all(x == '?' or (x == y) for x, y in zip(h1, h2))

def fulfills(hypothesis, example):
    return all(h == '?' or h == e for h, e in zip(hypothesis, example))

def min_generalizations(h, x):
    new_h = list(h)
    for i in range(len(h)):
        if h[i] != x[i]:
            new_h[i] = '?'
    return [new_h]

def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == '?':
            for val in domains[i]:
                if val != x[i]:
                    new_h = h.copy()
                    new_h[i] = val
                    results.append(new_h)
        elif h[i] != x[i]:
            continue
        else:
            continue
    return results

def candidate_elimination(data):
    attributes = data.columns[:-1]
    domains = [list(data[attr].unique()) for attr in attributes]
    num_attributes = len(attributes)
    G = [['?'] * num_attributes]
    S = None

    for idx, row in data.iterrows():
        if row.iloc[-1] == 'Yes':
            S = [list(row[:-1])]
            break

    if S is None:
        S = [['∅ '] * num_attributes]

    for index, row in data.iterrows():
        example = list(row[:-1])
        label = row.iloc[-1]

        if label == 'Yes':
            G = [g for g in G if fulfills(g, example)]
            S_new = []
            for s in S:
                if fulfills(s, example):
                    S_new.append(s)
                else:
                    generalized = min_generalizations(s, example)
                    generalized = [h for h in generalized if any(more_general(g, h) for g in G)]
                    S_new.extend(generalized)

            S = []
            for hyp in S_new:
                if not any(more_general(hyp2, hyp) for hyp2 in S):
                    S = [h for h in S if not more_general(h, hyp)]
                    S.append(hyp)

        else:
            S = [s for s in S if not fulfills(s, example)]
            G_new = []
            for g in G:
                if fulfills(g, example):
                    specialized = min_specializations(g, domains, example)
                    specialized = [h for h in specialized if any(more_general(h, s) for s in S)]
                    G_new.extend(specialized)
                else:
                    G_new.append(g)

            G = []
            for hyp in G_new:
                if not any(more_general(hyp, hyp2) for hyp2 in G):
                    G = [h for h in G if not more_general(h, hyp)]
                    G.append(hyp)

    return S, G

def main():
    file_path = input("Enter the path of your CSV file: ")
    try:
        data = load_data(file_path)
        S, G = candidate_elimination(data)
        print("\nFinal Specific Hypotheses (S):")
        for s in S:
            print(s)
        print("\nFinal General Hypotheses (G):")
        for g in G:
            print(g)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Enter the path of your CSV file: /content/candidate.csv

Final Specific Hypotheses (S):
['Sunny', 'Warm', '?', 'Strong', '?', '?']

Final General Hypotheses (G):
['Sunny', '?', '?', '?', '?', '?']
['?', 'Warm', '?', '?', '?', '?']
